In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
keys = ['train', 'valid', 'test']

In [3]:
p, q, a, e = {}, {}, {}, {}
for k in keys :
    file = open('CBTest/data/cbtest_NE_' + k + '.txt').read().strip().split('\n\n')
    file = [x.split('\n') for x in file]
    file = [[y.lower().split()[1:] for y in x] for x in file]
    p[k] = [" ".join([" ".join(y) for y in x[:-1]]) for x in file]
    qa = [x[-1] for x in file]
    q[k] = [" ".join(x[:-2]) for x in qa]
    a[k] = [x[-2] for x in qa]
    e[k] = [x[-1].split('|') for x in qa]

In [4]:
from vectorizer import Vectorizer

In [26]:
vec = Vectorizer(min_df=1)

In [27]:
vec.fit(p['train'] + q['train'])

In [34]:
answers = set([y for k in e for x in e[k] for y in x])

In [38]:
vec.entity2idx = {}
i = 0
for w in answers :
    vec.entity2idx[w] = i
    i += 1
vec.idx2entity = {v:k for k, v in vec.entity2idx.items()}

In [47]:
def generate_label_and_filter(elist, a) :
    entities = np.zeros((len(vec.entity2idx),))
    for w in elist :
        entities[vec.entity2idx[w]] = 1
    
    return entities, vec.entity2idx[a]

In [52]:
vec.paragraphs = {}
vec.questions = {}
vec.entity_masks = {}
vec.answers = {}

for k in keys :
    vec.paragraphs[k] = vec.texts_to_sequences(p[k])
    vec.questions[k] = vec.texts_to_sequences(q[k])
    vec.entity_masks[k] = []
    vec.answers[k] = []
    for i in range(len(p[k])) :
        mask, answer = generate_label_and_filter(e[k][i], a[k][i])
        vec.entity_masks[k].append(mask)
        vec.answers[k].append(answer)

In [57]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
vectors = Vectors('wiki.simple.vec', url=url)

In [58]:
vec.word_dim = vectors.dim
import numpy as np
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [59]:
import pickle
pickle.dump(vec, open('CBTest/vec_cbt.p', 'wb'))